In [1]:
# 1-(b) Provide an estimate with M=63, 
# bias가 있음 : 실제 예측치는 7.33 보다 작아야 함(discrete하게 모니터링 하기 때문)
# 윗쪽으로 bias 발생

import numpy as np

np.random.seed(1)
N=int(5*1e5)

T = 0.25  # 3 months
M = 63 # 3개월을 데일리 샘플링
dt = T/M

S0 = 50; sig=0.3; r=0.05; K=60; H=55; 

W = np.zeros((N,M)) #  N by M matrix
St = np.zeros((N,M))
t = np.zeros(M) 


for j in range(M):
    dW = np.random.randn(N)*np.sqrt(dt) # 분산이 sqrt(dt)인 정규분포 Y
    
    if j<1:
        W[:,j] = dW
    else:
        W[:,j] = W[:,j-1] + dW
        
    t[j] = dt*(j+1)
    St[:,j] = S0*np.exp((r-sig*sig/2)*t[j]+sig*W[:,j]) # St의 모든열 중 j번째 행에 S0*exp를 대입
    
St_max = np.max(St, axis=1) # 행(row)을 고정(axis=1), 각 시나리오별 63일동안 가장 높은(max)값 vector 추출
#print(np.size(St_max)) # dimension 확인

UOP_T = (K - St[:,-1])*(St[:,-1]<K)*(St_max<H) # 만기 pay-off: 마지막날의 종가(stock price)가 strike price(K)보다 작고, barrier(H) hitting이 없어야 함
UOP_0 = np.exp(-r*T)*UOP_T # 초기 값 (할인적용)

print("95% CI = [", np.mean(UOP_0)-1.96*np.std(UOP_0)/np.sqrt(N), \
     ", ", np.mean(UOP_0)+1.96**np.std(UOP_0)/np.sqrt(N), "]")

95% CI = [ 7.336207263557495 ,  7.633073755343082 ]


In [3]:
# 1-(c) consider "first hitting time" with rebate $5
# 최초로 barrier 넘어 갔을 때 종가시점으로 기록
# present value 기준으론 조금 손해 (??)
# (b)케이스의 unfair한 에러를 조금 줄였지만, 정확한 simulation이라 할 순 없음
# 어떤 방향의 어느정도 bias인지 평가할 순 없음

import numpy as np

np.random.seed(1)
N=int(5*1e5)

T = 0.25  # 3 months
M = 63 # 3개월을 데일리 샘플링
dt = T/M

S0 = 50; sig=0.3; r=0.05; K=60; H=55; 

# with rebate
rebate = 5

W = np.zeros((N,M)) #  matrix
St = np.zeros((N,M))
t = np.zeros(M) 

# 시나리오 별(N) 변수(벡터) 2개 추가
hitting_idx = np.zeros(N) # 처음에 50부터 시작이니 zero로 채움
first_hitting_time = np.zeros(N) # first hitting만 기록

for j in range(M):
    dW = np.random.randn(N)*np.sqrt(dt) # 분산이 sqrt(dt)인 정규분포 Y
    
    if j<1:
        W[:,j] = dW
    else:
        W[:,j] = W[:,j-1] + dW
        
    t[j] = dt*(j+1)
    St[:,j] = S0*np.exp((r-sig*sig/2)*t[j]+sig*W[:,j]) # St의 모든열 중 i+1번째 행에 S0*exp를 대입
    
    # (중요) hitting idx = 1 경우 + 기존 hitting idx = 0 & 지금 생성된 주식가격 > H 일때 기록
    first_hitting_time = t[j]*(1-hitting_idx)*(St[:,j]>=H) + first_hitting_time*hitting_idx 
    
    # (중요) hitting event발생시, idx 업데이트 
    hitting_idx = (1-hitting_idx)*(St[:,j]>=H) + hitting_idx 
    
#UOP_T = (K - St[:,-1])*(St[:,-1]<K)*(St_max<H)

# hitting idx=1 일때는 rebate 받고 옵션 행사 or hitting idx=0일때는 만기 pay-off 지급
UOP_0 = np.exp(-r*T)*(K-St[:,-1])*(St[:,-1]<K)*(1-hitting_idx) + np.exp(-r*first_hitting_time)*rebate*hitting_idx       # 만기까지 가서 pay-off 받거나, 중간에 쳐서 rebate 받거나
        

print("95% CI = [", np.mean(UOP_0)-1.96*np.std(UOP_0)/np.sqrt(N), \
     ", ", np.mean(UOP_0)+1.96*np.std(UOP_0)/np.sqrt(N), "]")

95% CI = [ 9.738646697073676 ,  9.770132219913911 ]
